In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')

# print(openai.Engine.list())

global_path = "../../lib/StaticDataStorage/data/Global.json"


def load_global_static_data() -> dict:
    with open(global_path, "r") as f:
        globalJson = json.load(f)
    
    return globalJson

def get_terms(globalJson: dict) -> list:
    ret = []

    for item in globalJson['terms']:
        termDef = item['termDef']
        term = termDef.split(" - ")[0]
        ret.append(term)

    return ret

def get_terms_string() -> str:
    globalJson = load_global_static_data()
    terms = get_terms(globalJson)
    return ", ".join(terms)

def print_lines(text, max_line_length=120):
    lines = []
    line = ""
    for word in text.split():
        if len(line + word) > max_line_length:
            lines.append(line)
            line = ""
        line += word + " "
    lines.append(line)

    for line in lines:
        print(line)

class ChatEngine:
    model = ""
    temp = 0

    def __init__(self, model="gpt-3.5-turbo", temp=0):
        self.model = model
        self.temp = temp

class Chat:
    messages: list = []
    engine: ChatEngine

    def __init__(self, engine, system_message, user_messages = []):
        self.engine = engine
        self.messages.append({"role": "system", "content": system_message})
        self.messages.extend([{"role": "user", "content": message} for message in user_messages])

    def get_answer(self, user_message = ""):
        if user_message != "":
            self.messages.append({"role": "user", "content": user_message})
        response = openai.ChatCompletion.create(
            model=self.engine.model,
            messages=self.messages,
            temperature=self.engine.temp,
        )
        answer = response.choices[0].message["content"]
        self.messages.append({"role": "assistant", "content": answer})
        return answer


In [ ]:
import json

continue_term_prompt = """You are a terms helper robot \
your task is to guess, based on the list of terms, \
which one term, not in the list, \
is the most relevant to already given terms."""

def predict_next_term():
    engine = ChatEngine(temp=0.2)
    chat = Chat(engine, continue_term_prompt)
    terms_str = get_terms_string()
    answer = chat.get_answer(terms_str)
    print_lines(answer)

predict_next_term()


In [ ]:
give_definition_prompt = """
You are a terms definition robot \
your task is to give a best possible definition to the user requested term. \

You should give a definition step by step, according to the this rules: \

1. Definition should contain only information about what the term intrinsically is, \
without any information about what the term is not, and what the term is related to.
Every entity that can be removed from definition without changing the meaning, should be removed.
2. Definition should based on the existing list of terms (list would be provided to you).
3. Check if there exist terms that also fit this definition and are not synonymous to the term.
4. Definition should be as simple as possible.
5. Definition should be as short as possible. \

List of existing terms: '''{0}''' \

User requested definition for term: '''{1}''' \

Format of answer should be: \
Raw term definition: <try to define user term for the first time. format 'term - definition'> <new line> \

Text of first rule: <rewrite first rule here> <new line> \
Thoughts about rule: <your thoughts on how to apply thie first rule> <new line> \
Updated term: <rewritten definition after applying first rule in format 'term - definition'> <new line> \

Text of second rule: <rewrite second rule here> <new line> \
Thoughts about rule: <your thoughts on how to apply thie second rule> <new line> \
Updated term: <rewritten definition after applying second rule in format 'term - definition'> <new line> \

Text of third rule: <rewrite third rule here> <new line> \
Thoughts about rule: <your thoughts on how to apply thie third rule> <new line> \
Updated term: <rewritten definition after applying third rule in format 'term - definition'> <new line> \

Text of fourth rule: <rewrite fourth rule here> <new line> \
Thoughts about rule: <your thoughts on how to apply thie fourth rule> <new line> \
Updated term: <rewritten definition after applying fourth rule in format 'term - definition'> <new line> \

Text of fifth rule: <rewrite fifth rule here> <new line> \
Thoughts about rule: <your thoughts on how to apply thie fifth rule> <new line> \
Updated term: <rewritten definition after applying fifth rule in format 'term - definition'> <new line> \
"""

def give_definition(new_term):
    terms_str = get_terms_string()

    engine = ChatEngine(temp=0.2)
    chat = Chat(engine, give_definition_prompt.format(terms_str, new_term))

    answer = chat.get_answer()
    print_lines(answer)

give_definition("Energy")

In [12]:
def add_term(term, definition, area=""):
    globalJson = load_global_static_data()

    termDef = '{0} - {1}'.format(term, definition)
    if area != "":
        globalJson['terms'].append({"area": area, "termDef": termDef})
    else:
        globalJson['terms'].append({"termDef": termDef})

    with open(global_path, "w") as f:
        json.dump(globalJson, f, indent=4)


add_term("Energy", f"scalar physical quantity, which is a single {{measure}} of various forms of {{motion}} and interaction of {{matter}}", "phys")